In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#Install dependencies
#!pip install -q tensorflow tensorflow_hub tensorflow_io kaggle

#Importing necessary modules
from google.colab import files
import zipfile
import pandas as pd
import os
from google.colab import files

#Preprocessing modules:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
import numpy as np
from sklearn.preprocessing import LabelEncoder

#Model modules:
import librosa
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_hub as hub

#Evaulation metrics:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")


2026-02-23 04:49:14.730648: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771822154.941941      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771822155.006462      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771822155.486138      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771822155.486181      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771822155.486185      55 computation_placer.cc:177] computation placer alr

In [3]:
# Generating the labelled csv file since the dataset does not come with one:
import os
import pandas as pd

wav_dir = "/kaggle/input/insectsound1000/InsectSound1000"


species_to_category = {
    "Palomena_prasina": "pest",
    "Nezara_viridula": "pest",
    "Halyomorpha_halys": "pest",
    "Rhaphigaster_nebulos": "pest",
    "Bradysia_difformis": "pest",
    "Myzus_persicae": "pest",
    "Trialeurodes_vaporariorum": "pest",
    "Tuta_absoluta": "pest",

    "Bombus_terrestris": "beneficial",
    "Episyrphus_balteatus": "beneficial",
    "Coccinella_septempunctata": "beneficial",
    "Aphidoletes_aphidimyza": "beneficial"
}

data = []

for filename in os.listdir(wav_dir):
    if filename.endswith(".wav"):
        parts = filename.split("_")

        if len(parts) >= 3:
            species = parts[1] + "_" + parts[2]

            if species in species_to_category:
                category = species_to_category[species]

                full_path = os.path.join(wav_dir, filename)

                data.append({
                    "filepath": full_path,
                    "label": species,
                    "category": category
                })

df = pd.DataFrame(data)

csv_path = "/kaggle/working/insect_labels_binary.csv"
df.to_csv(csv_path, index=False)

print("CSV saved at:", csv_path)
print("Total samples:", len(df))
print("\nCategory distribution:\n")

#Converting "category" column to binary values for easy processing

df["category"] = df["category"].map({
    "pest": 0,
    "beneficial": 1
})

#Displaying the category value spread
print(df["category"].value_counts())




CSV saved at: /kaggle/working/insect_labels_binary.csv
Total samples: 165982

Category distribution:

category
0    102076
1     63906
Name: count, dtype: int64


In [4]:
# Label file
metadata = pd.read_csv("/kaggle/working/insect_labels_binary.csv")
metadata.head()

,filepath,label,category
0,/kaggle/input/insectsound1000/InsectSound1000/...,Episyrphus_balteatus,beneficial
1,/kaggle/input/insectsound1000/InsectSound1000/...,Halyomorpha_halys,pest
2,/kaggle/input/insectsound1000/InsectSound1000/...,Nezara_viridula,pest
3,/kaggle/input/insectsound1000/InsectSound1000/...,Nezara_viridula,pest
4,/kaggle/input/insectsound1000/InsectSound1000/...,Bradysia_difformis,pest


In [5]:
#Directory verification
fp = "/kaggle/input/insectsound1000/InsectSound1000"
audio = os.listdir(fp)
print(f"Total files: ",len(audio))
print("Sample files: ", audio[:5])

Total files:  165982
Sample files:  ['2023613-16-37_Episyrphus_balteatus_000064_s71_ch0.wav', '2022524-16-36_Halyomorpha_halys_000038_s48_ch0.wav', '2022512-17-17_Nezara_viridula_000056_s2_ch0.wav', '202268-16-20_Nezara_viridula_000030_s15_ch0.wav', '2022329-13-41_Bradysia_difformis_000021_s13_ch0.wav']


In [6]:
# Extracting labels and filepaths

X = []
y = []

species_to_category = {
    "Palomena_prasina": 0,
    "Nezara_viridula": 0,
    "Halyomorpha_halys": 0,
    "Rhaphigaster_nebulos": 0,
    "Bradysia_difformis": 0,
    "Myzus_persicae": 0,
    "Trialeurodes_vaporariorum": 0,
    "Tuta_absoluta": 0,

    "Bombus_terrestris": 1,
    "Episyrphus_balteatus": 1,
    "Coccinella_septempunctata": 1,
    "Aphidoletes_aphidimyza": 1
}

for filename in os.listdir(wav_dir):
    if filename.endswith(".wav"):
        parts = filename.split("_")

        if len(parts) >= 3:
            species = parts[1] + "_" + parts[2]

            if species in species_to_category:
                full_path = os.path.join(wav_dir, filename)

                X.append(full_path)
                y.append(species_to_category[species])

print("Total files:", len(X))
print("Example:", X[0], "→", y[0])


Total files: 165982
Example: /kaggle/input/insectsound1000/InsectSound1000/2023613-16-37_Episyrphus_balteatus_000064_s71_ch0.wav → 1


In [7]:
# Verification of sampling rate to make sure it says as 16k
import librosa

file_path = X[0]  # first audio file
_, sr = librosa.load(file_path, sr=None)

print("Sampling rate:", sr)


Sampling rate: 16000


In [8]:
# Encoding labels for compatability with model
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)

num_classes = len(le.classes_)

print("Number of classes:", num_classes)
print("Encoded example:", y_encoded[:5])
print("Class mapping:")
for i, cls in enumerate(le.classes_):
    print(i, "→", cls)


Number of classes: 2
Encoded example: [1 0 0 0 0]
Class mapping:
0 → 0
1 → 1


In [9]:
#Initializing to NumPy arrays for better performance
X = np.array(X)
y_encoded = np.array(y_encoded)
print(X[:5],"\n",y_encoded[:5])

['/kaggle/input/insectsound1000/InsectSound1000/2023613-16-37_Episyrphus_balteatus_000064_s71_ch0.wav'
 '/kaggle/input/insectsound1000/InsectSound1000/2022524-16-36_Halyomorpha_halys_000038_s48_ch0.wav'
 '/kaggle/input/insectsound1000/InsectSound1000/2022512-17-17_Nezara_viridula_000056_s2_ch0.wav'
 '/kaggle/input/insectsound1000/InsectSound1000/202268-16-20_Nezara_viridula_000030_s15_ch0.wav'
 '/kaggle/input/insectsound1000/InsectSound1000/2022329-13-41_Bradysia_difformis_000021_s13_ch0.wav'] 
 [1 0 0 0 0]


In [ ]:
# Function to load audio waveform from each file 

import soundfile as sf
import tensorflow as tf
import numpy as np

def load_mono_py(file_path):
    file_path = file_path.numpy().decode("utf-8")
    data, sr = sf.read(file_path)

    if len(data.shape) > 1:
        data = np.mean(data, axis=1)
    if sr != 16000:
        data = librosa.resample(data, orig_sr = sr, target_sr = 16000);

    return data.astype(np.float32)

def load_mono_tf(file_path):
    waveform = tf.py_function(
        load_mono_py,
        [file_path],
        tf.float32
    )
    waveform.set_shape([None])  
    return waveform


yamnet = hub.load('https://tfhub.dev/google/yamnet/1')

# Program to extract the relevant embeddings from the waveforms in the previous fn


def extembed(file_path, label):
    waveform = load_mono_tf(file_path)
    
    scores, embeddings, spectrogram = yamnet(waveform)
    embedding = embeddings

    # # Average temporal embeddings
    # mean = tf.reduce_mean(embeddings, axis=0) 
    # #emean = tf.reduce_max(embeddings, axis=0) #In case mean does not work
    # std = tf.math.reduce_std(embeddings, axis=0)
    # embedding = tf.concat([mean, std], axis=0)

    return embedding, label


In [ ]:
num = len(set(y_encoded))

AUTOTUNE = tf.data.AUTOTUNE

dataset = tf.data.Dataset.from_tensor_slices((X, y_encoded)) # Converts arrays into TensorFlow datasets

dataset = (
    dataset
    .shuffle(10000)
    .map(extembed, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(512)
    .prefetch(AUTOTUNE)
)


In [ ]:
# Classification phase: Dense NN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Main dataset split
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y_encoded,
    test_size=0.2,
    stratify=y_encoded,
    random_state=42
)

#Standardizing inputs to improve performance:

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)


#Training and validation datasets:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))

AUTOTUNE = tf.data.AUTOTUNE

train_ds = (
    train_ds
    .shuffle(10000)
    .map(extembed, num_parallel_calls=AUTOTUNE)
    .batch(512)
    .prefetch(AUTOTUNE)
)

val_ds = (
    val_ds
    .map(extembed, num_parallel_calls=AUTOTUNE)
    .batch(512)
    .prefetch(AUTOTUNE)
)


In [ ]:
# CNN Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(None, 1024)),

    tf.keras.layers.Conv1D(512, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv1D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
# Code to handle class imbalance
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)

class_weight_dict = dict(zip(classes, class_weights))
count = 0;
for k,v in class_weight_dict.items():
    print("Key: ",k,"\n","Value: ",v)
    count += 1;
    if count > 5: break;

In [ ]:
print(y_train[:5])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
#Adding callbacks - Why?
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        "best_model.h5",
        save_best_only=True
    )
]


In [ ]:
# import soundfile as sf

# for f in X[:1000]:
#     try:
#         info = sf.info(f)
#         print(f"{f}: channels={info.channels}, subtype={info.subtype}")
#     except Exception as e:
#         print(f"Corrupt or unreadable: {f} ({e})")


In [ ]:
 #Model Training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    class_weight=class_weight_dict,
    callbacks=callbacks
)





In [ ]:
#Model evaluation
val_loss, val_acc = model.evaluate(val_ds)
print("Validation Accuracy:", val_acc)


In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_true = []
y_pred = []

for embeddings, labels in val_ds:
    preds = model.predict(embeddings)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

print(classification_report(y_true, y_pred))


In [ ]:
cm = confusion_matrix(y_val, y_pred)
cls = le.classes_
labels = np.unique(np.concatenate((y_val, y_pred)))
comb = [cls[i] for i in labels]

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=comb, yticklabels=comb)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Load the saved model and label encoder
with open("yamnet_rf_model.pkl", "rb") as f:
    mod = pickle.load(f)

with open("label_encoder.pkl", "rb") as f:
    le = pickle.load(f)

def pred_sound(file_path):

    wf = audioprep(file_path)
    embed = extembed(wf)
    embed = embed.reshape(1, -1)
    predicted_encoded = mod.predict(embed)
    predicted_label = le.inverse_transform(predicted_encoded)
    return predicted_label[0]


Re-initializing dataset with noise: